In [1]:
##########################################################################
# Created on Sat Nov 20 15:21:45 2021                                    #
# Python for Financial Analysis and Risk Management                      #
# @author: Meng Lipeng (FRM, CFA)                                        #
##########################################################################

# 14.1.2.Merton Model

$V_0$:Enterprise value at $T_0$(Bond initial date)\
$V_T$:Enterprise value at $T$(Bond maturity date)\
$E_0$:Stock value at $T_0$\
$E_T$:Stock value at $T$\
$D$:Par value of zero rate bond\
$\sigma_V$:Volatiltiy of enterprise value, assumed constant\
$\sigma_E$:Instantaneous volatiltiy of stock return

$$E_T=max(V_T-D,0)\tag{14-1}$$\
$$E_0=V_0N(d_1)-De^{-rT}N(d_2)\tag{14-2}$$\
where,\
$$d_1=\frac{ln(V_0/D)+(r+\sigma_V ^2/2)T}{\sigma_V \sqrt{T}}$$
$$d_2=\frac{ln(V_0/D)+(r-\sigma_V ^2/2)T}{\sigma_V \sqrt{T}}=d_1-\sigma_V\sqrt{T}$$

$N(d_2)$ is probability of exercise when option expires in risk neutral state, thus probability of non default of risk neutral enterprises at time T is also $N(d_2)$, probability of default of risk neutral enterprises at time T is $1-N(d_2)=N(-d_2)$

According to Ito's lemma,\
$$\sigma_E E_0=N(d_1)\sigma_V V_0\tag{14-3}$$

Simultaneous equations(14-1) and (14-3), we can get $V_0$ and $\sigma_V$. Finally, the risk neutral default probability $N(-d_2)$ can be obtained.

# 14.1.3.Measure probability of default

In [2]:
import pandas as pd
import numpy as np

price_Sun=pd.read_excel('Sun stock price daily(201208-201402).xlsx',engine='openpyxl',sheet_name="Sheet1",header=0,index_col=0)
return_Sun=np.log(price_Sun/price_Sun.shift(1))
sigma_Sun=np.sqrt(252)*np.std(return_Sun)
sigma_Sun=float(sigma_Sun)
print('Annualized return voloatility of Sun is ',round(sigma_Sun,4))

Annualized return voloatility of Sun is  0.4238


In [3]:
equity=21.85
debt=63.90
tenor=1
rate=0.050001

def f(x):
    from numpy import exp,log,sqrt
    from scipy.stats import norm
    V,sigma_V=x
    d1=(log(V/debt)+(rate+pow(sigma_V,2)/2)*tenor)/(sigma_V*sqrt(tenor))
    d2=d1-sigma_V*sqrt(tenor)
    eq1=V*norm.cdf(d1)-debt*exp(-rate*tenor)*norm.cdf(d2)-equity #equation(14-2)
    eq2=sigma_Sun*equity-norm.cdf(d1)*sigma_V*V #equation(14-3)
    return(eq1,eq2)

import scipy.optimize as sco

result=sco.fsolve(func=f,x0=[80,0.5])
print('Enterprise value of Sun at 2014/02/19 is ',round(result[0],4))
print('Volatiltiy of enterprise value is ',round(result[-1],4))

Enterprise value of Sun at 2014/02/19 is  82.6259
Volatiltiy of enterprise value is  0.1124


In [4]:
def PD_Merton(E,D,V,sigma,r,T):
    '''Define a function to calculate default probability of an enterprise.
    E:Stock value
    D:Par of bond
    V:Enterprise value
    sigma:Annualized volatility of enterprise value
    r:Risk free rate
    T:Tenor of bond'''
    from numpy import log,sqrt
    from scipy.stats import norm
    
    d1=(log(V/D)+(r+pow(sigma,2)/2)*T)/(sigma*sqrt(T))
    d2=d1-sigma*sqrt(T)
    PD=norm.cdf(-d2)
    return PD

In [5]:
PD_Sun=PD_Merton(E=equity,D=debt,V=result[0],sigma=result[-1],r=rate,T=tenor)
print('Probability of default of Sun at 2014/02/19 is ',round(PD_Sun,6))

Probability of default of Sun at 2014/02/19 is  0.003725


In [6]:
equity_new=70.75
debt_new=28.02
rate_new=0.052378
sigma_new=0.4654

def g(x):
    from numpy import exp,log,sqrt
    from scipy.stats import norm
    V,sigma_V=x
    d1=(log(V/debt_new)+(rate_new+pow(sigma_V,2)/2)*tenor)/(sigma_V*sqrt(tenor))
    d2=d1-sigma_V*sqrt(tenor)
    eq1=V*norm.cdf(d1)-debt_new*exp(-rate_new*tenor)*norm.cdf(d2)-equity_new 
    eq2=sigma_new*equity_new-norm.cdf(d1)*sigma_V*V
    return(eq1,eq2)

result_new=sco.fsolve(func=g,x0=[80,0.5])
print('Enterprise value of Sun at 2011/12/30 is ',round(result_new[0],4))
print('Volatiltiy of enterprise value is ',round(result_new[-1],4))

Enterprise value of Sun at 2011/12/30 is  97.3399
Volatiltiy of enterprise value is  0.3383


In [7]:
PD_Sun_new=PD_Merton(E=equity_new,D=debt_new,V=result_new[0],sigma=result_new[-1],r=rate_new,T=tenor)
print('Probability of default of Sun at 2011/12/30 is ',round(PD_Sun_new,6))

Probability of default of Sun at 2011/12/30 is  0.000123


In [8]:
M=PD_Sun/PD_Sun_new
print('Probability of default of Sun at 2014/02/19 is ', round(M,2), 'times that at 2011/12/30')

Probability of default of Sun at 2014/02/19 is  30.35 times that at 2011/12/30
